In [2]:
#################################################################################################################
#####                                   002-2c-Energy_Correlation            ####################################
#################################################################################################################
####  This step is to create a different heatmap with specific correlation marix for each kind of energy
####   to analyze the properties correlated with each type of energy
####
#### Identify different energies in the df_Clean. 
#### Creata a new and different df for each energy thanks to a loop. 
#### Create the correlation matrix and then select only pairs realted to our target variable. ("Co2_wltp") 
#### Call the funtion to print the heatmap. 
#################################################################################################################

with open('CMatrixEnergy_'+V_Country+Target_Var+'.csv', 'w') as f:
    f.truncate(0)

Energies = df_Clean["Energy"].unique()
df_ = {}     

for z, Ene in enumerate(Energies):
    
# -- Skip electric vehicles as they don't have Co2_emisions.
    if Ene != 'electric' and  Ene != 'hydrogen':   
          
        df_Energy = df_Clean.loc[df_Clean["Energy"] == Ene ] 
 
        correlated_fields=[]    
        corr_matrix = pd.DataFrame(np.abs(df_Energy.select_dtypes(include=['int', 'float']).corr()))
        
        order = df_Energy.columns
        corr_matrix = corr_matrix.reindex(index=order, columns=order)
        corr_matrix = np.nan_to_num(corr_matrix)
        
        abs_corr_matrix = np.abs(corr_matrix)
        Submatrix = np.where(abs_corr_matrix)

# -- Loop to get pairs & corr Value for Relevant Fiel
        for i in range(len(Submatrix[0])):
            
            field1 = df_Clean.columns[Submatrix[0][i]]
            field2 = df_Clean.columns[Submatrix[1][i]]
              
            if field1 == Target_Var and  field2 != Target_Var:
                correlation_value = corr_matrix[Submatrix[0][i], Submatrix[1][i]]
                correlated_fields.append((field1, field2, round(correlation_value * 100))) 
                
# -- Sort fields to see all plots in same order.  
        correlated_fields = sorted(correlated_fields, key=lambda x: x[2], reverse=True)
        df_corr = pd.DataFrame(correlated_fields, columns=['Field 1', 'Field 2', 'Correlation Value'])
        corr_matrix = df_corr.pivot(index='Field 1', columns='Field 2', values='Correlation Value')
                
# -- draw the heatmap                
        fig = px.imshow(corr_matrix, color_continuous_scale='RdBu',text_auto=True)
        fig.update_layout( title='CO2_wltp Correlated fields for energy = '+ Ene + ' - ' + V_Country, title_x=0.4 )
        fig.update_traces(text=corr_matrix.values)
    
        display(fig) 
                   
# -- Store Correlation mat results in a file for next steps .         
        df_corr.insert(0,'Ene',Ene)
        df_corr = df_corr.sort_values(by='Correlation Value', ascending=False)

        with open('CMatrixEnergy_'+V_Country+Target_Var+'.csv', 'a') as f:
             df_corr.to_csv(f, header=f.tell()==0)



NameError: name 'V_Country' is not defined